In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transfroms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Set the random seed for reproducibility
SEED = 111
torch.manual_seed(SEED)
np.random.seed(SEED)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"GPU Available: {device}")



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/archive
!ls

In [ ]:

# Custom Dataset class for training
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

# DataFrame for training data
def generate_df(data_path):
    classes, class_paths = zip(*[(label, os.path.join(label, image))
                                for label in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, label))
                                for image in os.listdir(os.path.join(data_path, label))])

    df = pd.DataFrame({'Class Path': class_paths, 'Class': classes})
    return df

# training and testing data
TEST_DIR = '/content/gdrive/My Drive/archive/Testing'
TRAIN_DIR = '/content/gdrive/My Drive/archive/Training'

# Generate DataFrames
tr_df = generate_df(TRAIN_DIR)
ts_df = generate_df(TEST_DIR)

# Label encoding to assign numerical classes
label_mapping = {label: idx for idx, label in enumerate(tr_df['Class'].unique())}
tr_df['Class'] = tr_df['Class'].map(label_mapping)
ts_df['Class'] = ts_df['Class'].map(label_mapping)

# Data transformations
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomResizedCrop(32),
    T.ToTensor()
])

# Initialize the custom dataset
train_dataset = CustomImageDataset(tr_df, root_dir=TRAIN_DIR, transform=transform)
test_dataset = CustomImageDataset(ts_df, root_dir=TEST_DIR, transform=transform)

# Load data using DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 7))

# Plot the count of images per class
ax = sns.countplot(data=tr_df, y='Class', order=tr_df['Class'].value_counts().index)

plt.xlabel('')
plt.ylabel('')
plt.title('Count of Images in Training Data', fontsize=20)

# Add count labels to each bar
ax.bar_label(ax.containers[0])

# Show the plot
plt.show()


In [ ]:

warnings.filterwarnings("ignore")

plt.figure(figsize=(15, 7))

# Plot the count of images per class
ax = sns.countplot(data=ts_df, y='Class', order=ts_df['Class'].value_counts().index)

plt.xlabel('')
plt.ylabel('')
plt.title('Count of Images in Testing Data', fontsize=20)

# Add count labels to each bar
ax.bar_label(ax.containers[0])

# Show the plot
plt.show()


In [ ]:


# Parameters
image_size = (150, 150)
batch_size = 32


# Define data augmentation and normalization for training data
train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.15),  # Approximating brightness_range
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=12.5, shear=12.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define normalization only for testing data
test_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Get class indices and their corresponding names
class_indices_train = train_dataset.class_to_idx
class_indices_train_list = list(train_dataset.classes)

# Display the class indices for training data
print("Categorical types for the training data:")
print(class_indices_train)

# Image shape: height, width, RGB
image_shape = (image_size[0], image_size[1], 3)

# Training parameters
epochs = 40
steps_per_epoch = len(train_loader)
validation_steps = len(test_loader)

print(f'Image shape: {image_shape}')
print(f'Epochs: {epochs}')
print(f'Batch size: {batch_size}')
print(f'Steps Per Epoch: {steps_per_epoch}')
print(f'Validation Steps: {validation_steps}')


In [ ]:

N_TYPES = len(class_indices_train)

#cited from: https://www.kaggle.com/code/guslovesmath/cnn-brain-tumor-classification-99-accuracy the structure of CNN models
# Define the model architecture
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=3)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=3)

        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)

        # Calculate the size after flattening the last convolutional layer
        self.flatten_size = 128 * 2 * 2

        self.fc1 = nn.Linear(self.flatten_size, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(nn.ReLU()(self.conv1(x)))
        x = self.pool2(nn.ReLU()(self.conv2(x)))
        x = self.pool3(nn.ReLU()(self.conv3(x)))
        x = nn.ReLU()(self.conv4(x))
        x = torch.flatten(x, 1)
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return nn.Softmax(dim=1)(x)

# Instantiate the model and print its summary
model = ConvNet(num_classes=N_TYPES).to(device)
print(model)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.869, 0.995))

# Define the loss function
criterion = nn.CrossEntropyLoss()


In [ ]:
!pip install torchviz
# Define a dummy input tensor
dummy_input = torch.randn(1, 3, *image_size).to(device)

# produce a computation graph
output = model(dummy_input)

# Visualize the graph using torchviz
make_dot(output, params=dict(model.named_parameters())).render("model_architecture", format="png")


In [ ]:
# Initialize lists to collect training and validation statistics
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []

# training loop to collect statistics
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    running_train_correct = 0
    total_train_samples = 0

    # Training phase
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_train_correct += torch.sum(preds == labels)
        total_train_samples += labels.size(0)

    avg_train_loss = running_train_loss / len(train_loader)
    train_accuracy = running_train_correct.double() / total_train_samples
    train_loss_history.append(avg_train_loss)
    train_acc_history.append(train_accuracy.item())

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    running_val_correct = 0
    total_val_samples = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            running_val_correct += torch.sum(preds == labels)
            total_val_samples += labels.size(0)

    avg_val_loss = running_val_loss / len(test_loader)
    val_accuracy = running_val_correct.double() / total_val_samples
    val_loss_history.append(avg_val_loss)
    val_acc_history.append(val_accuracy.item())

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

# Plot training and validation statistics
_, ax = plt.subplots(ncols=2, figsize=(15, 6))

# Accuracy plot
ax[0].plot(train_acc_history)
ax[0].plot(val_acc_history)
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].legend(['Train', 'Validation'])
ax[0].grid(alpha=0.2)

# Loss plot
ax[1].plot(train_loss_history)
ax[1].plot(val_loss_history)
ax[1].set_title('Model Loss')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].legend(['Train', 'Validation'])
ax[1].grid(alpha=0.2)

plt.show()


In [ ]:

# Switch model to evaluation mode
model.eval()

# Collect all true labels and predicted labels
true_labels = []
predicted_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)

# Label formatting
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(ticks=np.arange(N_TYPES) + 0.5, labels=[name.title() for name in class_indices_train_list], ha='center')
plt.yticks(ticks=np.arange(N_TYPES) + 0.5, labels=[name.title() for name in class_indices_train_list], va='center')

plt.show()
